<a href="https://colab.research.google.com/github/johannesR17/predicciones-de-ventas-alimenticias/blob/main/predicciones_de_ventas1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CODING DOJO: Proyecto 1**
## Creado por: Johannes Rodríguez

* Fecha creado: 10/04/2023
* Fecha de actualización: 15/04/2023
* Descripción: este script perform my first portfolio.

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filename = '/content/drive/MyDrive/Python for Data Science/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


* Identifiquen el objetivo (X) y las características (y): Asignen la columna "Item_Outlet_Sales" como el objetivo y el resto de las variables relevantes como el matriz de características.  

* Realicen un train test split 

* Creen un pipeline de preprocesamiento para preparar el conjunto de datos para el aprendizaje automático.

In [4]:
# 1) Target Vector & Features Matriz

y = df['Item_Outlet_Sales']

X = df.drop(columns = 'Item_Outlet_Sales')

In [5]:
# 2) Simple imputer at train test split

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [7]:
df.shape

(8523, 12)

In [8]:
print(df.isna().sum().sum(), 'missing values')

3873 missing values


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.isna().any()

Item_Identifier              False
Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [10]:
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [11]:
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')

In [12]:
num_tuple = (median_imputer, num_selector)
cat_tuple = (freq_imputer, cat_selector)

col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')
col_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('simpleimputer-1',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7700>),
                                ('simpleimputer-2',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7970>)])

In [13]:
col_transformer.fit(X_train)

X_train_imputed = col_transformer.transform(X_train)
X_test_imputed = col_transformer.transform(X_test)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_train_imputed.isna().any()

Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [14]:
# 3) Pre-processing Pipeline

import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

Codificación ordinal

In [15]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [16]:
replacement_dictionary = {'Regular':1, 'reg':1, 'Low Fat':0, 'LF':0, 'low fat':0}
df['Item_Fat_Content'].replace(replacement_dictionary, inplace=True)
df['Item_Fat_Content']

0       0
1       1
2       0
3       1
4       0
       ..
8518    0
8519    1
8520    0
8521    1
8522    0
Name: Item_Fat_Content, Length: 8523, dtype: int64

In [17]:
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [18]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [19]:
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [20]:
# Numeric pipeline
numeric_pipe = make_pipeline(median_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [21]:
# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [22]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7700>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7970>)])

In [23]:
# fit on train
preprocessor.fit(X_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7700>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc39e9f7970>)])

In [24]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [25]:
# Checking the missing values and that the data is scaled and have a one-hot encoding
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')
X_train_processed

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6392, 1595)




array([[ 0.82748547, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.56664432, -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.12102782,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.12389588, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77599877, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82748547, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

* Instanciando modelos de regresión para este último apartado. 

1) 

* Creen un modelo de regresión lineal.
* Evalúen el rendimiento del modelo basándose en r^2.
* Evalúen el rendimiento del modelo basándose en RECM.

In [26]:
# Make a linear regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [27]:
# Fit train
reg.fit(X_train_processed,y_train)

LinearRegression()

In [28]:
# Pred. set training
train_preds = reg.predict(X_train_processed)
# Pred. set test
test_preds = reg.predict(X_test_processed)

In [29]:
# Evaluate the model on r^2
from sklearn.metrics import r2_score
R2_train = r2_score(y_train, train_preds)
print("Entrenamiento:",R2_train, "\n")
R2_test = r2_score(y_test, test_preds)
print("Prueba:",R2_test, "\n")

Entrenamiento: 0.6697187033205461 

Prueba: -2.5651505187750874e+20 



In [30]:
# Evaluate the model on RECM
from sklearn.metrics import mean_squared_error
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
print("Entrenamiento:",rmse_train, "\n")
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print("Prueba:",rmse_test, "\n")

Entrenamiento: 988.6621604756797 

Prueba: 26602992352564.258 



2)
* Creen un modelo simple de árbol de regresión.

* Comparen el rendimiento del modelo basándose en r^2.

* Comparen el rendimiento del modelo basándose en RECM.  

In [31]:
# Stating
from sklearn.tree import DecisionTreeRegressor
dec_tree = DecisionTreeRegressor(random_state = 42)

In [32]:
# Fit train
dec_tree.fit(X_train_processed, y_train)

DecisionTreeRegressor(random_state=42)

In [33]:
# Locate predict
train_preds = dec_tree.predict(X_train_processed)
test_preds = dec_tree.predict(X_test_processed)

In [34]:
# Evaluate the model on r^2
train_score = r2_score(y_train, train_preds)
test_score = r2_score(y_test, test_preds)
print("Entrenamiento:",train_score, "\n")
print("Prueba:",test_score, "\n")

Entrenamiento: 1.0 

Prueba: 0.2274416042770071 



In [35]:
# Evaluate the model on RECM
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
print("Entrenamiento:",rmse_train, "\n")
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print("Prueba:",rmse_test, "\n")

Entrenamiento: 4.96419825584149e-15 

Prueba: 1459.9558442994373 



**3)** 

¿Qué modelo implementar?

El modelo de árbol de regresión.

Justificación. - A diferencia del modelo de regresión lineal, el Regression tree describe datos cuantitativos mesurables: específicamente aplicando la evaluación en el coeficiente de determinación R2. La cuestión del modelo Regression Linear se rompe en este dataset, por lo que al aplicarse el onehotencoder: la columna "Item Identefier" se expande y terminas con un dataframe con un poco más de 8000 columnas.